In [1]:
reset()
from sage.combinat.rooted_tree import number_of_rooted_trees

In [2]:
def alonso_schott_tn(n):
    # This is Sage's function that generates the same values as the formula of equation (1) in Alonso/Schott paper,
    # and it computes the number of unlabeled rooted trees with n nodes.
    # Takes an integer n as input and returns the number of trees for every n value in form of a dictionay: key = n, value = number of trees
    # For example, n = 10, it returns {1: 1, 2: 1, 3: 2, 4: 4, 5: 9, 6: 20, 7: 48, 8: 115, 9: 286, 10: 719}
    D = {0:1}
    N = [number_of_rooted_trees(i) for i in range(n+1)]
    for i in range(1, n+1):
        D[i] = N[i]
    return D

In [3]:
def generate_all_jd_pairs_and_prob_distribution(n):
    # P is list of probabilities. The list will automatically be normalised if sum(P) is not equal to 1.
    # It generates a probability distribution where the probability of selecting x is P[x]
    L = [] # stores tuple (j, d)
    t = alonso_schott_tn(n)
    for d in range(1, n):
        for j in range(1, floor((n-1)/d)+1):
            p = (d*t[n-j*d]*t[d])/((n-1)*t[n])
            L.append((j, d, p.n()))
    P = [e[2] for e in L]
    dist = GeneralDiscreteDistribution(P)
    return L, dist

In [4]:
def ranrut(n):
    if n == 1:
        tree = sage.combinat.rooted_tree.RootedTrees_size(1)[0]
        return tree
    elif n == 2:
        tree = sage.combinat.rooted_tree.RootedTrees_size(2)[0]
        return tree
    else:
        all_j_d_prob_tuples, our_dist = generate_all_jd_pairs_and_prob_distribution(n)
        x = our_dist.get_random_element() # randomly selected index
        one_jdpair_and_prob_tuple = all_j_d_prob_tuples[x]
        j = one_jdpair_and_prob_tuple[0]
        d = one_jdpair_and_prob_tuple[1]
        tree = ranrut(n-j*d) # T'
        Tpp = ranrut(d)      # T''
        for i in range(j):
            tree = tree.graft_on_root(Tpp)
    return tree

In [5]:
def vertex_remover(A, B):
    # removes all vertices in A, contained in B
    # and returns the list A
    return [v for v in A if v not in B]

In [6]:
def get_subtree_vertices(D, v):
    # for any given vertex v in directed tree D,
    # it returns all vertices in the subree of v, including v as list
    return list(D.breadth_first_search(start=v))

In [7]:
def find_largest_subtree_at_level(D, V):
    # for list of vertices V at a level in directed tree D,
    # this function finds the largest subtree starting at that level
    # and retuns the starting vertex and vertices of its subtree
    # as a tuple (v0, [v0,v1,v2,....]) - note that list contains starting vertex

    max_size = -1
    max_subtree = []

    for v in V:
        L = get_subtree_vertices(D, v)
        if len(L) > max_size:
            max_size = len(L)
            max_subtree = L
    return max_subtree

In [8]:
def find_largest_subtree_at_level_improved(D, V):
# f o r l i s t of v e r t i c e s V at a l e v e l in d i r e c t e d t r e e D,
# t h i s function f i n d s the l a r g e s t subtree s t a r t i n g at that l e v e l
# and retuns the s t a r t i n g vertex and v e r t i c e s of i t s subtree
# as a tuple ( v0 , [ v0 , v1 , v2 , . . . . ] ) - note that l i s t contains
#s t a r t i n g vertex
# t h i s function ” attempts ” to s o l v e s the tie - breaking problem \
#where subtrees have the same
# s i z e but but d i f f e r e n t structure , and choses subtree with the
# l a r g e s t degree to break t i e s . For example :
# __o____
# / /
# o o
# | / /
# o o o
# |
# o
# here the r i g h t node at l e v e l 1 i s chosen because such \
#s e l e c t i o n
# saves more nodes ( only 2 burn ) , rather than choosing l e f t node\at
# l e v e l 1 in which case 3 nodes would have burned .

    Mtmp = []                                          # w i l l be constructed as [ subtree_size , degree , L]
    for v in V:
        L = get_subtree_vertices(D, v )
        deg = len(D.neighbors_out(v))
        Mtmp.append([len(L),deg, L])
    M = sorted (Mtmp, key = lambda x : ( x [ 0 ] , x [ 1 ] ) ) 
                                                          # sorted accord . \to subtree_size f i r s t then degree
    if len (M) == 0 :
        return []
    else:
        return M[- 1][- 1]                                               # returns l a s t element ' s subtree

In [9]:
def greedy_alg(D):
    # input argument D is the tree (as directed graph)
    burned = []
    saved = []
    unprocessed = D.vertices(sort = True) # unprocessed nodes (neither burned or saved)

    root = unprocessed[0] # root (1) is always the first element due to sort
    burned.append(root)
    unprocessed = unprocessed[1:]

    L = D.level_sets()
    for level in range(1, len(L)): # 1 because root node is lit and we save at levels below it
        level_nodes = L[level]
        level_nodes = vertex_remover(level_nodes, saved)
        level_nodes = vertex_remover(level_nodes, burned)
        lsubtree = find_largest_subtree_at_level(D, level_nodes)
        if len(unprocessed) != 0:
            saved += lsubtree
            level_nodes.remove(lsubtree[0])# remove saved subtree root from level so that it can be saved
            burned += level_nodes
            unprocessed = vertex_remover(unprocessed, saved)
            unprocessed = vertex_remover(unprocessed, burned)
    return burned, saved

In [10]:
def greedy_alg_improved (D) :
# input argument D i s the t r e e ( as d i r e c t e d graph )
    burned = []
    saved = []
    unprocessed = D.vertices(sort = True)                               # unprocessed nodes (\n e i t h e r burned or saved )
    root = unprocessed[0]                                             # root (1) i s always the f i r s t element due\to s o r t
    burned.append(root)
    unprocessed = unprocessed[1:]
    L = D.level_sets()
    for level in range(1,len(L)):                                # 1 because root node i s l i t and \we save at l e v e l s below i t
        level_nodes = L[level]
        level_nodes = vertex_remover(level_nodes,saved)
        level_nodes = vertex_remover(level_nodes,burned)
        lsubtree = find_largest_subtree_at_level_improved(D,level_nodes)
        if len(lsubtree) > 0 :
            if len(unprocessed) != 0 :
                saved += lsubtree
                level_nodes.remove(lsubtree[0])                          # remove saved subtree root from l e v e l so that i t can be saved
                burned += level_nodes
                unprocessed = vertex_remover (unprocessed,saved)
                unprocessed = vertex_remover (unprocessed,burned)
    return burned,saved

In [11]:
def number_of_root_degree_two_and_only_two_nodes_burn_trees(n, num_of_runs):
    cnt = 0
    for i in range(num_of_runs):
        t = ranrut(n)
        ot = OrderedTree(t) # changing the order tree so that we can use Sage graph-theory functions
        ot_as_graph = ot.canonical_labelling().to_undirected_graph()
        D = DiGraph(ot_as_graph.edges(sort = True))
        L = D.neighbors_out(1) # counting the number of neighbors of the root, always labeled 1
        if len(L) == 2:
            b,s = greedy_alg(D)
            if len(b) == 2:
                cnt += 1
    return (cnt/num_of_runs * alonso_schott_tn(n)[n]).n()

In [12]:
def number_of_root_degree_two_and_only_two_nodes_burn_trees_improved(n , num_of_runs ) :
    cnt = 0
    for i in range(num_of_runs):
        t = ranrut(n)
        ot = OrderedTree(t)                      # changing the order t r e e so that we can use Sage graph - theory f u n c t i o n s
        ot_as_graph = ot.canonical_labelling().to_undirected_graph()
        D = DiGraph(ot_as_graph.edges(sort = True))
        L = D.neighbors_out(1)                                             # counting the number of neighbors of the root, always l a b e l e d 1
        if len(L) == 2 :
            b,s = greedy_alg_improved (D)
            if len(b) == 2 :
                cnt += 1
    return (cnt/num_of_runs*alonso_schott_tn(n)[n]).n()


In [13]:
def degree_two_and_two_nodes_burn_counter(n, num_of_runs):
    cnt = 0
    for i in range(num_of_runs):
        cnt += number_of_root_degree_two_and_only_two_nodes_burn_trees(n, num_of_runs)
    return (cnt/num_of_runs).n()

In [14]:
def degree_two_and_two_nodes_burn_counter_improved(n, num_of_runs):
    cnt = 0
    for i in range(num_of_runs):
        cnt +=  number_of_root_degree_two_and_only_two_nodes_burn_trees_improved(n, num_of_runs)
    return (cnt/num_of_runs).n()

In [19]:
degree_two_and_two_nodes_burn_counter(25, 100)

5.21961597862500e8

In [20]:
degree_two_and_two_nodes_burn_counter(25, 100)

5.19067553359500e8

In [21]:
degree_two_and_two_nodes_burn_counter(25, 100)

5.22375032791500e8

In [22]:
degree_two_and_two_nodes_burn_counter_improved(25, 100)

5.16173508856500e8

In [23]:
5.16173508856500e8/_tn(25)

0.249700000000000

In [24]:
5.19067553359500e8/_tn(25)

0.251100000000000

In [15]:
# This is our theoretical function that you can use to compare with
# the results of the two functions immediately above.
def our_formula(n):
    s = 0
    for k in range(ceil((n-1)/2), n-1):
        s += alonso_schott_tn(k)[k]*alonso_schott_tn(n-k-2)[n-k-2]
    if is_odd(n):
        s+= -(_tn((n-3)/2)*(_tn((n-3)/2)-1))/2
    return s

In [27]:
for k in range(4, 21):
    print(k, ",", our_formula(k))

4 , 1
5 , 3
6 , 6
7 , 15
8 , 33
9 , 84
10 , 201
11 , 519
12 , 1296
13 , 3413
14 , 8791
15 , 23481
16 , 61659
17 , 166753
18 , 444887
19 , 1214510
20 , 3277696


In [22]:
for k in range(2, 7):
    print(k, ", ", degree_two_and_two_nodes_burn_counter(k, 300), ", ", degree_two_and_two_nodes_burn_counter_improved(k, 300), "," , our_formula(k))

2 ,  0.000000000000000 ,  0.000000000000000 , 0


KeyboardInterrupt: 

In [26]:
degree_two_and_two_nodes_burn_counter(20, 300)

3.25173382528889e6

In [16]:
def _tn(n):
    return alonso_schott_tn(n)[n]

In [18]:
print(alonso_schott_tn(10))

{0: 1, 1: 1, 2: 1, 3: 2, 4: 4, 5: 9, 6: 20, 7: 48, 8: 115, 9: 286, 10: 719}


In [24]:
(our_formula(4)/_tn(4)).n()

0.250000000000000

In [28]:
(our_formula(5)/_tn(5)).n()

0.333333333333333

In [29]:
(our_formula(6)/_tn(6)).n()

0.300000000000000

In [30]:
(our_formula(7)/_tn(7)).n()

0.312500000000000

In [31]:
(our_formula(8)/_tn(8)).n()

0.286956521739130

In [25]:
(our_formula(9)/_tn(9)).n()

0.293706293706294

In [26]:
(our_formula(10)/_tn(10)).n()

0.279554937413074

In [34]:
(our_formula(11)/_tn(11)).n()

0.285016286644951

In [35]:
(our_formula(12)/_tn(12)).n()

0.271926143516576

In [36]:
(our_formula(13)/_tn(13)).n()

0.276229376902130

In [25]:
(our_formula(25)/_tn(25)).n()

0.252391657986885

In [17]:
(our_formula(1000)/_tn(1000)).n()

0.229465487694098

In [21]:
(our_formula(2000)/_tn(2000)).n()

0.229195129530362

In [22]:
(our_formula(3000)/_tn(3000)).n()

0.229104763498634

In [18]:
(our_formula(4000)/_tn(4000)).n()

0.229059515999185

In [24]:
(our_formula(5000)/_tn(5000)).n()

0.229032342320442

In [43]:
(our_formula(7000)/_tn(7000)).n()

0.229001259279325

In [44]:
(our_formula(8000)/_tn(8000)).n()

0.228991538936569

In [25]:
(our_formula(9000)/_tn(9000)).n()

0.228983976113218

In [27]:
(our_formula(14000)/_tn(14000)).n()

0.228962354089033

In [28]:
 (our_formula(16000)/_tn(16000)).n()

0.228957485889210

In [29]:
(our_formula(18000)/_tn(18000)).n()

0.228953698566083

In [30]:
(our_formula(20000)/_tn(20000)).n()

0.228950668071291

In [31]:
(our_formula(25000)/_tn(25000)).n()

0.228945211636975

In [32]:
(our_formula(27000)/_tn(27000)).n()

0.228943594505252

In [ ]:
for k in range(66,101):
    print(our_formula(1000*k)/_tn(1000*k).n())


0.228931642779966
0.228931519206684
0.228931399266091
0.228931282800308
0.228931169660476
0.228931059706117
0.228930952804557
0.228930848830389
0.228930747664981
0.228930649196027
0.228930553317130
0.228930459927418
0.228930368931189
0.228930280237588
0.228930193760297
0.228930109417264
0.228930027130435
0.228929946825517
0.228929868431754
0.228929791881719
0.228929717111119
0.228929644058616
0.228929572665662


In [20]:
T = sage.combinat.rooted_tree.RootedTrees_size(10)
for t in T:
    ascii_art(t)
    print(ascii_art(t))
    print("_____________________")

o
|
o
|
o
|
o
|
o
|
o
|
o
|
o
|
o
|
o
_____________________
  o
  |
  o
  |
  o
  |
  o
  |
  o
  |
  o
  |
  o
  |
  o_
 / /
o o
_____________________
  o
  |
  o
  |
  o
  |
  o
  |
  o
  |
  o
  |
  o_
 / /
o o
  |
  o
_____________________
   o
   |
   o
   |
   o
   |
   o
   |
   o
   |
   o
   |
  _o__
 / / /
o o o
_____________________
  o
  |
  o
  |
  o
  |
  o
  |
  o
  |
  o_
 / /
o o
  |
  o
  |
  o
_____________________
   o
   |
   o
   |
   o
   |
   o
   |
   o
   |
  _o__
 /   / 
o   o_
   / /
  o o
_____________________
  o
  |
  o
  |
  o
  |
  o
  |
  o
  |
  o_
 / /
o o
| |
o o
_____________________
   o
   |
   o
   |
   o
   |
   o
   |
   o
   |
  _o__
 / / /
o o o
    |
    o
_____________________
    o
    |
    o
    |
    o
    |
    o
    |
    o
    |
  __o___
 / / / /
o o o o
_____________________
  o
  |
  o
  |
  o
  |
  o
  |
  o_
 / /
o o
  |
  o
  |
  o
  |
  o
_____________________
   o
   |
   o
   |
   o
   |
   o
   |
  _o__
 /   / 
o   o
    |


In [ ]:
##find limit analytically of our forumla/ alonso schott which is t_n in RANRUT
##(fraction of trees of 2 nodes burning) - difficult for n terms 
##(fraction of 1 nodes which is 1/3)
